In [1]:
import gzip
import math
import numpy
import random
import sklearn
import string
from collections import defaultdict
from nltk.stem.porter import *
from sklearn import linear_model
from gensim.models import Word2Vec
import dateutil
from scipy.sparse import lil_matrix # To build sparse feature matrices, if you like
import pandas as pd

import html
import random
import json

In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

This takes a 1.5 mins to run

In [3]:
# allRatings = [l for l in parse("ratebeer.json.gz")] # sampled has 300,000 reviews
ratings = [l for l in parse("sampled_ratebeer.json.gz")][0] # sampled has 300,000 reviews

# Sample and compress dataset (ALREADY RAN DONT RUN)

In [ ]:
# # Random sample the dataset because it's too big

# sampleSize = 300000
# sampledAllRatings = random.sample(ratings, sampleSize)

In [ ]:
# import json

# with open("sampled_ratebeer.json", "w") as f:
#     json.dump(sampledAllRatings, f)

In [ ]:
# import gzip
# import shutil

# # File paths
# input_file = 'sampled_ratebeer.json'      # Your existing JSON file
# output_file = 'sampled_ratebeer.json.gz'  # Desired compressed file

# # Compress the JSON file
# with open(input_file, 'rb') as f_in:
#     with gzip.open(output_file, 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

# print(f"Compressed {input_file} to {output_file}")


Compressed sampled_ratebeer.json to sampled_ratebeer.json.gz


# Clean data

Check missing labels (style)

In [4]:
for rating in ratings:
    if 'beer/style' in rating:
        rating['beer/style'] = html.unescape(rating['beer/style'])

In [5]:
styles = set()
no_style = []

for rating in ratings:
    if 'beer/style' in rating:
        styles.add(rating['beer/style'])
    else:
        no_style.append(rating)

len(styles), len(no_style)

(89, 0)

In [6]:
allKeys = set(ratings[0].keys())

Check missing features

In [7]:
hasMissingData = []

for rating in ratings:
    if allKeys != set(rating.keys()):
        hasMissingData.append(rating)
assert(len(hasMissingData) == 0)

## Change string ratings to numerical values

In [8]:
ratings[0]

{'beer/name': 'Dogfish Head Midas Touch Golden Elixir',
 'beer/beerId': '7293',
 'beer/brewerId': '198',
 'beer/ABV': '9',
 'beer/style': 'Traditional Ale',
 'review/appearance': '3/5',
 'review/aroma': '6/10',
 'review/palate': '4/5',
 'review/taste': '6/10',
 'review/overall': '15/20',
 'review/time': '1172188800',
 'review/profileName': 'chef201atc',
 'review/text': 'Clear copper pour with a white bubbly head that quickly fades.  Nose is sweet, fruity, candi sugar.  Medium bodied with average carbonation.  Flavor is slightly astrigent, a litlle bitter, some of the fruitiness comes through.'}

In [9]:
# Changes the string rating (X/Y) to decimal
def scoreToInt(score: string) -> float:
    numer, denom = score.split('/')
    return int(numer) / int(denom)

In [10]:
# Change all ratings to have numerical scores
scoreNames = ['review/appearance', 'review/aroma', 'review/palate', 'review/taste',  'review/overall']

for rating in ratings:
    for name in scoreNames:
        rating[name + '_numerical'] = scoreToInt(rating[name])

## Remove the escape characters from the text

In [ ]:
# https://stackoverflow.com/questions/8115261/how-to-remove-all-the-escape-sequences-from-a-list-of-strings

escapeChars = ''.join([chr(char) for char in range(1, 32)])
x = "testing \tsdfsdf \t"

translator = str.maketrans('', '', escapeChars)
x.translate(translator)


['t',
 'e',
 's',
 't',
 'i',
 'n',
 'g',
 ' ',
 '\t',
 's',
 'd',
 'f',
 's',
 'd',
 'f',
 ' ',
 '\t']

# Split dataset

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [12]:
splitIndex = 200000
ratingsTrain = ratings[:splitIndex]
ratingsTest = ratings[splitIndex:]

# Bag of words!

In [13]:
ratingsTrain

[{'beer/name': 'Dogfish Head Midas Touch Golden Elixir',
  'beer/beerId': '7293',
  'beer/brewerId': '198',
  'beer/ABV': '9',
  'beer/style': 'Traditional Ale',
  'review/appearance': '3/5',
  'review/aroma': '6/10',
  'review/palate': '4/5',
  'review/taste': '6/10',
  'review/overall': '15/20',
  'review/time': '1172188800',
  'review/profileName': 'chef201atc',
  'review/text': 'Clear copper pour with a white bubbly head that quickly fades.  Nose is sweet, fruity, candi sugar.  Medium bodied with average carbonation.  Flavor is slightly astrigent, a litlle bitter, some of the fruitiness comes through.',
  'review/appearance_numerical': 0.6,
  'review/aroma_numerical': 0.6,
  'review/palate_numerical': 0.8,
  'review/taste_numerical': 0.6,
  'review/overall_numerical': 0.75},
 {'beer/name': 'Lost Abbey Red Barn Ale',
  'beer/beerId': '64608',
  'beer/brewerId': '7043',
  'beer/ABV': '6.7',
  'beer/style': 'Saison',
  'review/appearance': '4/5',
  'review/aroma': '8/10',
  'review/pa